In [ ]:
! pip install llama-index
! pip install azure-search-documents==11.4.0b8 --pre

In [1]:
import nest_asyncio

nest_asyncio.apply()

import os
import keys

from llama_index import ServiceContext
from llama_index.llms import AzureOpenAI
from llama_index.schema import MetadataMode

from llama_index.embeddings import OpenAIEmbedding

In [2]:
llm = AzureOpenAI(
    engine="raidGPT",
    model="gpt-4",
    temperature=0.0,
    api_base="https://raid-ses-openai.openai.azure.com/",
    api_key=keys.gpt_key,
    api_type="azure",
    api_version="2023-05-15"
)

emb_llm = OpenAIEmbedding(
    engine="swiftfaq-ada002",
    model="text-embedding-ada-002",
    temperature=0.0,
    api_base="https://raid-ses-openai.openai.azure.com/",
    api_key=keys.gpt_key,
    api_type="azure",
    api_version="2023-05-15"
)

In [10]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader('../data/124').load_data()

## Integration with Azure Cognitive Search

In [3]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

import keys

from llama_index.vector_stores.cogsearch import (
    IndexManagement,
    CognitiveSearchVectorStore,
    MetadataIndexFieldType
)

from llama_index import (
    LangchainEmbedding,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    VectorStoreIndex,
)

service_endpoint = "https://aimlexplorationsearch.search.windows.net"
index_name = "rsaf-cognitive-search"
key = keys.cognitive_key
credential = AzureKeyCredential(key)

## Creating own nodes from azure output

In [4]:
from llama_index.query_engine import CustomQueryEngine
from llama_index.retrievers import BaseRetriever
from llama_index.response_synthesizers import get_response_synthesizer, BaseSynthesizer
from llama_index.schema import Node, NodeWithScore
from llama_index import QueryBundle
from typing import List

In [39]:
from llama_index.chat_engine import CondenseQuestionChatEngine, ContextChatEngine

class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both semantic search and hybrid search."""

    def __init__(
        self,
        search_client : SearchClient,
    ) -> None:
        """Init params."""
        self._search_client = search_client
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        nodes = []
        # filtering
        results = self._search_client.search(search_text=query_bundle, top=3)
        # citations management
        # possible for rights management (post filtering)
        for i in results:
            nodes.append(NodeWithScore(node=Node(text=i["content"]), score=i['@search.score']))

        return nodes
 
search_client = SearchClient(endpoint=service_endpoint, index_name="chat-demo", credential=credential)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=emb_llm)
    
chat_engine = ContextChatEngine.from_defaults(retriever=CustomRetriever(search_client=search_client), verbose=True, service_context=service_context)

chat_engine.chat("What are advanced transitions?")

AgentChatResponse(response='Advanced transitions in helicopter operations refer to specific techniques used for landing and taking off from challenging conditions such as confined spaces, high altitudes, or at a high All-Up Weight (AUW). These techniques are employed when the basic transition technique is not feasible due to limitations in the engine power available or availability of forward distance. The objectives of advanced transitions are to determine the power margin available through hover and forward flight power checks, select the appropriate advanced transition technique, conduct a Out of Ground Effect (OGE), In Ground Effect (IGE), cushion creep and running take off transition to the climb, and conduct a transition from forward flight to an OGE hover, zero speed and running landing.', sources=[ToolOutput(content='system: Context information is below.\n--------------------\nOFFICIAL (CLOSED)   \n13-1 \nOFFICIAL (CLOSED)   PAR T B \n \nCHAPTER 13  \n \nADVANCED TRANSITIONS  \

### API friendly version

### This creates a new llamaindex compatible Azure Search Index

In [5]:
client_index = SearchIndexClient(endpoint=service_endpoint, credential=credential)

In [7]:
def create_azure_index(index_client, index_name : str, metadata_fields : dict, llm, emb_llm, filepath) -> str:
    
    """
    Spins up an azure cognitive search index.
    Will delete any index named as such so BE CAREFUL
    """
    
    vector_store = CognitiveSearchVectorStore(
        search_or_index_client=index_client,
        index_name=index_name,
        filterable_metadata_field_keys=metadata_fields,
        index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
        id_field_key="id",
        chunk_field_key="content",
        embedding_field_key="embedding",
        metadata_string_field_key="li_jsonMetadata",
        doc_id_field_key="li_doc_id",
    )
    
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    service_context = ServiceContext.from_defaults(llm=llm, embed_model=emb_llm)
    
    documents = SimpleDirectoryReader(filepath).load_data()
    
    ####################################
    # Chunking/Form Intelligence here  #
    ####################################
    
    
    index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context, service_context=service_context
)
    
    return "{}".format(index_name) + " created"

In [ ]:
create_azure_index(index_client=client_index, index_name="chat-demo", metadata_fields={"page_label" : ("page_label", MetadataIndexFieldType.INT32)}, 
                   llm = llm, emb_llm=emb_llm, filepath="../data/124")

In [11]:
vector_store = CognitiveSearchVectorStore(
    search_or_index_client=client_index,
    index_name="chat-demo",
    filterable_metadata_field_keys={},
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="content",
    embedding_field_key="embedding",
    metadata_string_field_key="li_jsonMetadata",
    doc_id_field_key="li_doc_id",
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=emb_llm)

index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context, service_context=service_context
)


### Testing custom filters

In [19]:
index_name = "quickstart"

metadata_fields = {
    "page_label": "page_label",
    "theme": "theme",
    "director": "director",
}

client_index = SearchIndexClient(endpoint=service_endpoint, credential=credential)

vector_store = CognitiveSearchVectorStore(
    search_or_index_client=client_index,
    index_name=index_name,
    filterable_metadata_field_keys=metadata_fields,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="content",
    embedding_field_key="embedding",
    metadata_string_field_key="li_jsonMetadata",
    doc_id_field_key="li_doc_id",
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=emb_llm)

index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context, service_context=service_context
)

In [14]:
from llama_index.schema import TextNode

nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
        },
    ),
]

index.insert_nodes(nodes)

### This creates a custom retriever with a chat engine

In [ ]:
from llama_index.chat_engine import CondenseQuestionChatEngine, ContextChatEngine

class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both semantic search and hybrid search."""

    def __init__(
        self,
        search_client : SearchClient,
    ) -> None:
        """Init params."""
        self._search_client = search_client
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        nodes = []
        # filtering
        results = self._search_client.search(search_text=query_bundle, top=3)
        # citations management
        # possible for rights management (post filtering)
        for i in results:
            nodes.append(NodeWithScore(node=Node(text=i["content"]), score=i['@search.score']))

        return nodes
 
search_client = SearchClient(endpoint=service_endpoint, index_name="chat-demo", credential=credential)
    
chat_engine = ContextChatEngine.from_defaults(retriever=CustomRetriever(search_client=search_client), verbose=True, service_context=service_context)

chat_engine.chat("What are advanced transitions?")

### This allows you to query that index

In [200]:
# Create a custom class that calls on search client
class AzureQueryEngine(CustomQueryEngine):
    """Azure Custom Query."""

    search_client : SearchClient
    response_synthesizer: BaseSynthesizer
    
    def custom_query(self, query_str: str):
        
        nodes = []
        # filtering
        results = self.search_client.search(search_text=query_str, top=3)
        # citations management
        # possible for rights management (post filtering)
        for i in results:
            nodes.append(NodeWithScore(node=Node(text=i["content"]), score=i['@search.score']))

        response_obj = self.response_synthesizer.synthesize(query_str, nodes)
        return response_obj

In [ ]:
def create_azure_query_engine(service_endpoint, llm, emb_llm):
    
    service_context = ServiceContext.from_defaults(llm=llm, embed_model=emb_llm)

    search_client = SearchClient(endpoint=service_endpoint, index_name="llamaindex-demo", credential=credential)
    
    synthesizer = get_response_synthesizer(response_mode="compact", service_context=service_context)
    query_engine = AzureQueryEngine(search_client=search_client, response_synthesizer=synthesizer)

    return query_engine